In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
original_df = pd.read_csv('newyork_housing.csv')

/tmp/ipykernel_3443197/1156967300.py:1: DtypeWarning: Columns (2,5,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,305,313,32

In [3]:
original_df.shape

(75630, 1507)

In [4]:
df = original_df.copy()

In [5]:
# remove all school features
df = df[df.columns.drop(list(df.filter(regex='schools')))]

# get school combined indices dataset
schools_df = pd.read_csv('census_schools_dataset.csv').iloc[:, 1:]

# concatanate school indices to original dataset
df = pd.concat([df, schools_df], axis=1)

In [6]:
# remove all photo url columns
df = df[df.columns.drop(list(df.filter(regex='photos')))]

# add number of photos attribute
number_of_photos = np.sum(pd.isnull(original_df.filter(like='photos', axis=1).values)==False, axis=1)
df['number_of_photos'] = number_of_photos

In [7]:
df = df.dropna(subset=['price']).reset_index(drop=True)

In [8]:
# remove all listings outside of NY
rows_to_drop = np.where(df['address/state'].values != 'NY')[0]
df = df.drop(df.index[rows_to_drop]).reset_index(drop=True)

# remove all columns that look or found unrelevant
list_of_columns_to_drop = [
    'address/state',
    'address/streetAddress',
    'currency',
    'resoFactsStats/parcelNumber',
    'resoFactsStats/atAGlanceFacts/0/factValue',
    'resoFactsStats/atAGlanceFacts/1/factValue',
    'resoFactsStats/atAGlanceFacts/2/factValue',
    'resoFactsStats/atAGlanceFacts/3/factValue',
    'resoFactsStats/atAGlanceFacts/4/factValue',
    'resoFactsStats/atAGlanceFacts/5/factValue',
]
'''
    'schools/0/link',
    'schools/0/name',
    'schools/0/totalCount',
    'schools/1/link',
    'schools/1/name',
    'schools/1/totalCount',
    'schools/2/link',
    'schools/2/name',
    'schools/2/totalCount',
    'schools/0/isAssigned',
    'schools/2/type',
    'schools/0/grades',
    'schools/1/grades',
    'schools/2/grades',
'''

for column_name in list_of_columns_to_drop:
    df = df.drop(column_name, axis=1)

In [9]:
# remove all columns with more than 33.3% missing values
percent_missing_threshold = 100/3

percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True)

columns_to_drop = missing_value_df.loc[missing_value_df['percent_missing'] > percent_missing_threshold].values[:, 0].tolist()

# Keep price history variables
for x in ['event', 'postingIsRental', 'price', 'priceChangeRate', 'time']:
    regex = re.compile(r'priceHistory/./' + x)
    columns_to_drop = [i for i in columns_to_drop if not regex.search(i)]
    
columns_to_drop.remove('resoFactsStats/atAGlanceFacts/6/factValue') # Days on Zillow
#print(columns_to_drop)
df = df.drop(columns_to_drop, axis=1)

In [10]:
# Collapse events
#print(df['priceHistory/0/event'].unique())
regex = re.compile(r'priceHistory/./event')
event = [i for i in df.columns if regex.search(i)]
event_name = list(df[event].apply(pd.value_counts).sum(axis = 1).index)
for x in event_name:
    df['number_of_' + x] = np.sum(df[event] == x, axis=1)
#print(np.sum(df[event] == 'Sold', axis=1), df['number_of_Sold'])

In [11]:
# add description length and exist attributes and remove description column

description_lengths = np.zeros((df['description'].shape[0],), dtype=int)

for i, description in enumerate(df['description'].values.tolist()):
    if pd.isnull(df['description'][i]) == True:
        continue
    if '\n' in description:
        description = description.replace('\n', ' ')
        df['description'][i] = description
        description_lengths[i] = len(description.split(' '))+1

description_exists = description_lengths>0

df['description_exists'] = description_exists
df['description_lengths'] = description_lengths
df.drop('description', axis=1, inplace=True)

/tmp/ipykernel_3443197/93139849.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['description'][i] = description


In [12]:
# remove all columns with unnecessary priceHistory information and keep only priceChangeRate

df = df[df.columns.drop(list(df.filter(regex='priceHistory')))]
df['priceChangeRate'] = original_df['priceHistory/0/priceChangeRate']

In [13]:
# Numeric days on Zillow
#df['days_on_Zillow'] = df['resoFactsStats/atAGlanceFacts/6/factValue'].str.extract('(\d+)').fillna(0).astype(int)
df['days_on_Zillow'] = df['resoFactsStats/atAGlanceFacts/6/factValue'].str.extract('(\d+)')
df['days_on_Zillow'] = pd.to_numeric(df['days_on_Zillow'])
print(df['days_on_Zillow'], df['resoFactsStats/atAGlanceFacts/6/factValue'])
df = df.drop('resoFactsStats/atAGlanceFacts/6/factValue', axis=1)

0         12.0
1        176.0
2        214.0
3        120.0
4         62.0
         ...  
75563      NaN
75564      NaN
75565      NaN
75566      NaN
75567      NaN
Name: days_on_Zillow, Length: 75568, dtype: float64 0         12 Days
1        176 Days
2        214 Days
3        120 Days
4         62 Days
           ...   
75563         NaN
75564         NaN
75565         NaN
75566         NaN
75567         NaN
Name: resoFactsStats/atAGlanceFacts/6/factValue, Length: 75568, dtype: object


In [14]:
for i, column_name in enumerate(df.columns[::-1]):
    current_column_values = df[column_name].values
    values, counts = np.unique(current_column_values.astype(str), return_counts=True)
    if np.max(counts) > 0.975*df.shape[0]:
        #print(column_name, counts)
        df = df.drop(column_name, axis=1)
    #else:
        #print(column_name, np.max(counts)/df.shape[0])

In [15]:
url_exists = np.zeros((df['url'].shape[0],), dtype=int)

for i, url in enumerate(df['url'].values.tolist()):
    if pd.isnull(df['url'][i]) == True:
        continue
    else:
        url_exists[i] = 1

df['url_exists'] = url_exists
df.drop('url', axis=1, inplace=True)

In [16]:
#all columns with sqft or acres - acres to sqft multiply the acre value by 43560
for i, column_name in enumerate(df.columns[::-1]):
    if (df[column_name].astype(str).str.contains(' Acres').any() or df[column_name].astype(str).str.contains(' sqft').any()) and column_name != 'description':
        print(column_name)
        Acres_rows = np.where(df[column_name].astype(str).str.contains(' Acres'))[0]
        df[column_name][Acres_rows] = df[column_name][Acres_rows].astype(str).str.replace(',','', regex=True).replace(' Acres','', regex=True).astype(float)*43560

        sqft_rows = np.where(df[column_name].astype(str).str.contains(' sqft'))[0]
        df[column_name][sqft_rows] = df[column_name][sqft_rows].astype(str).str.replace(',', '', regex=True).replace(' sqft', '', regex=True).astype(float)

resoFactsStats/lotSize


/tmp/ipykernel_3443197/177121190.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name][Acres_rows] = df[column_name][Acres_rows].astype(str).str.replace(',','', regex=True).replace(' Acres','', regex=True).astype(float)*43560
/tmp/ipykernel_3443197/177121190.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name][sqft_rows] = df[column_name][sqft_rows].astype(str).str.replace(',', '', regex=True).replace(' sqft', '', regex=True).astype(float)


resoFactsStats/livingArea


/tmp/ipykernel_3443197/177121190.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name][Acres_rows] = df[column_name][Acres_rows].astype(str).str.replace(',','', regex=True).replace(' Acres','', regex=True).astype(float)*43560
/tmp/ipykernel_3443197/177121190.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name][sqft_rows] = df[column_name][sqft_rows].astype(str).str.replace(',', '', regex=True).replace(' sqft', '', regex=True).astype(float)


In [17]:
bool_to_categories = []
for i, column_name in enumerate(df.columns[::-1]):
    if np.sum(np.where(df[column_name]==False)[0].size + np.where(df[column_name]==True)[0].size + np.where(df[column_name].values.astype('str')=='nan')[0].size)==df.shape[0]:
        bool_to_categories.append(column_name)
        df[column_name][np.where(df[column_name]==False)[0]] = 0
        df[column_name][np.where(df[column_name]==True)[0]] = 1

/tmp/ipykernel_3443197/873174732.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name][np.where(df[column_name]==False)[0]] = 0
/tmp/ipykernel_3443197/873174732.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name][np.where(df[column_name]==True)[0]] = 1
/tmp/ipykernel_3443197/873174732.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name][np.where(df[column_name]==False)[0]] = 0
/tmp/ipykernel

In [18]:
string_to_categories = [
    'address/city',
    'homeStatus',
    'resoFactsStats/atAGlanceFacts/0/factValue',
    'resoFactsStats/atAGlanceFacts/5/factLabel',
    'resoFactsStats/cityRegion',
    'resoFactsStats/homeType',
    'resoFactsStats/parkingFeatures/0',
]

'''
    'schools/0/level',
    'schools/1/level',
    'schools/2/level',
    'schools/0/grades',
    'schools/1/grades',
    'schools/2/grades'
'''


# factorize all string columns
for i, column_name in enumerate(df.columns[::-1]):
    if column_name in string_to_categories:
        #print(column_name, np.unique(df[column_name].values.astype('str')))
        if '/level' in column_name:
            df[column_name][np.where(df[column_name]=='Primary')[0]] = 0
            df[column_name][np.where(df[column_name]=='Elementary')[0]] = 1
            df[column_name][np.where(df[column_name]=='Middle')[0]] = 2
            df[column_name][np.where(df[column_name]=='High')[0]] = 3
        else:
            rows_to_factorize = np.where(df[column_name].isnull()==False)[0]
            df[column_name][rows_to_factorize] = pd.factorize(df[column_name][rows_to_factorize])[0]
        #print(df[column_name][rows_to_factorize])
        #print(pd.factorize(df[column_name][rows_to_factorize])[0])


/tmp/ipykernel_3443197/3166575509.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name][rows_to_factorize] = pd.factorize(df[column_name][rows_to_factorize])[0]
/tmp/ipykernel_3443197/3166575509.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name][rows_to_factorize] = pd.factorize(df[column_name][rows_to_factorize])[0]
/tmp/ipykernel_3443197/3166575509.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

In [19]:
df = df.loc[(df['homeStatus'] <= 2)]
df = df.loc[(df['price'] > 100)]

In [20]:
df.to_csv('processed_dataset.csv')